In [1]:
# Load system libraries
import sys
import datetime
import random

# Load ML libraries
import pandas

from numpy.random import shuffle

from pandas.plotting import scatter_matrix
from matplotlib import pyplot

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC as SVMClassifier

from scipy.spatial import distance
	
def _main():
	if (_showingHelp()):
		_showHelp()
		exit(0)
		
	if (_showingVersions()):
		_showVersions()
		
	# load dataset
	if (not _showingSummaryOnly()): print(datetime.datetime.now(), "explore_iris_7: Loading data");
	url = "file:/Users/bloop/dev/Decibels2/data_better.csv"
	# url = "file:///<file path here>/iris.csv"
	names = ['chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'label'];
	dataset = pandas.read_csv(url, names=names)

	if (_showingSamples()):
		_sampleData(dataset)
		_visualizeData(dataset, pyplot, scatter_matrix)

	# split data into train/test datasets
	if (not _showingSummaryOnly()): print(datetime.datetime.now(), "explore_iris_7: Splitting data into training and test sets");
	array = dataset.values
	X = array[:,0:25]
	X = X.astype('double')
	Y = array[:,25]
	Y = Y.astype('str')
	test_size = 0.20
	seed = 7
	X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

	# test options and perform evaluation metric
	seed = 7
	scoring = 'accuracy'
	models = []
	models.append(('Logistic Regression', LogisticRegression()))
	models.append(('KNN Neighbors', KNeighborsClassifier()))
	models.append(('Support Vector', SVMClassifier()))
	models.append(('DecisionTree', DecisionTreeClassifier()))
	models.append(('Random Forest', RandomForestClassifier()))
	
	if (not _showingSummaryOnly()):
		# evaluate models
		results = []
		names = []
		msg = ""
		for name, model in models:
			print(datetime.datetime.now(), "explore_iris_7: Evaluate model %s" % name);
			kfold = model_selection.KFold(n_splits=10, random_state=seed)
			cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
			results.append(cv_results)
			names.append(name)
			msg += "%20s: \t\t%f\t(%f)" % (name, cv_results.mean(), cv_results.std()) + "\n"
		print(msg)
		print();
		
	if ((_showingEval())):
		# compare algorithms
		fig = pyplot.figure()
		fig.suptitle('Compare algorithms')
		ax = fig.add_subplot(111)
		pyplot.boxplot(results)
		ax.set_xticklabels(names)
		pyplot.show()

	# make predictions on test dataset
	lor = LogisticRegression()
	_train(lor, "Logistic Regression", X_train, Y_train, X_test, Y_test)
	_predict(lor, "Logistic Regression", X_train, Y_train, X_test, Y_test)
	
	knn = KNeighborsClassifier()
	_train(knn, "KNeighbors Classifier", X_train, Y_train, X_test, Y_test)
	_predict(knn, "KNeighbors Classifier", X_train, Y_train, X_test, Y_test)

	svm = SVMClassifier()
	_train(svm, "Support Vector", X_train, Y_train, X_test, Y_test)
	_predict(svm, "Support Vector", X_train, Y_train, X_test, Y_test)

	dtc = DecisionTreeClassifier()
	_train(dtc, "Decision Tree Classifier", X_train, Y_train, X_test, Y_test)
	_predict(dtc, "Decision Tree Classifier", X_train, Y_train, X_test, Y_test)

	rfc = RandomForestClassifier()
	_train(rfc, "Random Forest", X_train, Y_train, X_test, Y_test)
	_predict(rfc, "Random Forest", X_train, Y_train, X_test, Y_test)

	myrnd = myRNDClassifier()
	_train(myrnd, "My Random", X_train, Y_train, X_test, Y_test)
	_predict(myrnd, "My Random", X_train, Y_train, X_test, Y_test)

	myknn = myKNNClassifier()
	_train(myknn, "My KNN", X_train, Y_train, X_test, Y_test)
	_predict(myknn, "My KNN", X_train, Y_train, X_test, Y_test)

	models.clear()
	models.append(('Logistic Regression', lor))
	models.append(('KNN Neighbors', knn))
	models.append(('Support Vector', svm))
	models.append(('DecisionTree', dtc))
	models.append(('Random Forest', rfc))
	models.append(('My random', myrnd))
	models.append(('My KNN', myknn))
	
	_predictionAccuracySummary(models, X_train, Y_train, X_test, Y_test)

#####################################################
# My random classifier
class myRNDClassifier:
	def fit(self, X_train, y_train):
		self.X_train = X_train
		self.y_train = y_train
		
	def predict(self, X_test):
		predictions = []
		for row in X_test:
			label = random.choice(self.y_train)
			predictions.append(label)
			
		return predictions

#####################################################
# My KNN K =1 classifier
class myKNNClassifier:
	def fit(self, X_train, y_train):
		self.X_train = X_train
		self.y_train = y_train
		
	def predict(self, X_test):
		predictions = []
		for row in X_test:
			label = self.closest(row)
			predictions.append(label)
		return predictions
	
	def closest(self, row):
		best_distance = euc(row, self.X_train[0])
		best_index = 0
		for i in range(1, len(self.X_train)):
			dist = euc(row, self.X_train[i])
			if (dist < best_distance):
				best_distance = dist
				best_index = i
				
		return(self.y_train[best_index])

def euc(a,b):
	return(distance.euclidean(a,b))
			
#####################################################
# training and prediction functions
def _train(alg, algName, X_train, Y_train, X_test, Y_test):
	if (not _showingSummaryOnly()): print(datetime.datetime.now(), "Begin training: ", algName)
	alg.fit(X_train, Y_train)
	if (not _showingSummaryOnly()): print(datetime.datetime.now(), "End training: ", algName)

def _predict(alg, algName, X_train, Y_train, X_test, Y_test):
	if (not _showingSummaryOnly()): print(datetime.datetime.now(), "Begin prediction: ", algName)
	predictions = alg.predict(X_test)
	if (not _showingSummaryOnly()): print(datetime.datetime.now(), "End prediction: ", algName)
	if (not _showingSummaryOnly()): print()
	if (not _showingSummaryOnly()): print("%s: accuracy_score=%0.2f" % (algName, accuracy_score(Y_test, predictions)))
	if (not _showingSummaryOnly()): print(confusion_matrix(Y_test, predictions))
	if (not _showingSummaryOnly()): print()
	if (not _showingSummaryOnly()): print(classification_report(Y_test, predictions))
	if (not _showingSummaryOnly()): print()

def _predictionAccuracySummary(models, X_train, Y_train, X_test, Y_test):
	print("Algorithm\t\tAccuracy Score")
	for name, model in models:
		predictions = model.predict(X_test)
		print("%20s\t\t%0.2f" % (name, accuracy_score(Y_test, predictions)))
	
#####################################################
# data sampling and visualization functions
def _sampleData(dataset):
	# show shape, first 10 records, description of frame and its distribution
	print(dataset.shape)
	print(dataset.head(20))
	print(dataset.describe())

	# show class distribution and distribution by sepal-length
	print(dataset.groupby('class').size())
	print(dataset.groupby('sepal-length').size())

def _visualizeData(dataset, pyplot, scatter_matrix):
	# visualize data and draw box and whisker plots
	dataset.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
	pyplot.show()

	# show histograms and scatter plot matrix
	dataset.hist()
	pyplot.show()
	scatter_matrix(dataset)
	pyplot.show()
	
#####################################################
# helper functions
def _parseArgumets(arg):
	for i in range(1, len(sys.argv)):
		if (sys.argv[i] == arg):
			return 1

def _showingHelp():
	return(_parseArgumets("--help"))

def _showingSummaryOnly():
	return(_parseArgumets("--summaryonly"))

def _showingVersions():
	return(not _parseArgumets("--summaryonly") and _parseArgumets("--version"))

def _showingSamples():
    return(not _parseArgumets("--summaryonly") and _parseArgumets("--sample"))

def _showingEval():
    return(not _parseArgumets("--summaryonly") and _parseArgumets("--eval"))

def _showHelp():
	print("iris7_explore_7models: syntax iris7_explore_7models --version --sample --eval --summaryonly")
	print("--help: show this help message");
	print("--version: show version info for Python runtime and ML libraries");
	print("--sample: show sample data");
	print("--eval: show evaluation of algorithms")
	print("--summaryonly: show only a summary of algorithms and their accuracy scores")

def _showVersions():
	# check versions of Python runtime and ML libraries
	import sys
	print('Python: {}'.format(sys.version))

	import scipy
	print('scipy: {}'.format(scipy.__version__))

	import numpy
	print('numpy: {}'.format(numpy.__version__))

	import matplotlib
	print('matplotlib: {}'.format(matplotlib.__version__))

	import pandas
	print('pandas: {}'.format(pandas.__version__))

	import sklearn
	print('sklearn: {}'.format(sklearn.__version__))
	
_main()

2019-04-16 14:36:50.349929 explore_iris_7: Loading data
2019-04-16 14:36:50.367061 explore_iris_7: Splitting data into training and test sets
2019-04-16 14:36:50.369888 explore_iris_7: Evaluate model Logistic Regression


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed

2019-04-16 14:36:51.064366 explore_iris_7: Evaluate model KNN Neighbors
2019-04-16 14:36:51.095061 explore_iris_7: Evaluate model Support Vector


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

2019-04-16 14:36:51.486935 explore_iris_7: Evaluate model DecisionTree
2019-04-16 14:36:51.646897 explore_iris_7: Evaluate model Random Forest


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

 Logistic Regression: 		0.537500	(0.041079)
       KNN Neighbors: 		0.352500	(0.042866)
      Support Vector: 		0.081250	(0.025769)
        DecisionTree: 		0.438750	(0.046587)
       Random Forest: 		0.548750	(0.056306)


2019-04-16 14:36:51.923667 Begin training:  Logistic Regression
2019-04-16 14:36:51.997363 End training:  Logistic Regression
2019-04-16 14:36:51.997455 Begin prediction:  Logistic Regression
2019-04-16 14:36:52.005710 End prediction:  Logistic Regression

Logistic Regression: accuracy_score=0.52
[[ 9  0  1  0  0  3  2  0  1  2]
 [ 0 14  0  0  0  0  0  0  2  1]
 [ 3  0 11  1  1  0  0  0  1  0]
 [ 0  0  1 13  2  0  1  1  1  2]
 [ 0  1  4  3  3  2  1  2  4  1]
 [ 1  3  1  1  0 10  0  1  1  0]
 [ 1  0  0  0  1  0 14  0  0  0]
 [ 0  0  1  1  0  0  0 24  1  0]
 [ 0  0  1  3  8  1  0  0  5  0]
 [ 8  0  3  8  0  2  3  1  1  1]]

              precision    recall  f1-score   support

       blues       0.41      0.50      0.45        18
   classical       0.78      0.82      

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


2019-04-16 14:36:52.147141 End training:  Random Forest
2019-04-16 14:36:52.147256 Begin prediction:  Random Forest
2019-04-16 14:36:52.150850 End prediction:  Random Forest

Random Forest: accuracy_score=0.54
[[13  0  0  0  0  1  2  0  1  1]
 [ 0 14  0  0  0  1  0  0  2  0]
 [ 7  0  6  0  1  1  0  0  2  0]
 [ 1  0  0 10  3  0  1  1  1  4]
 [ 1  0  0  6  9  2  0  0  2  1]
 [ 3  3  1  1  0  8  0  1  1  0]
 [ 1  0  0  0  0  0 15  0  0  0]
 [ 0  0  5  1  0  1  0 20  0  0]
 [ 0  0  4  2  3  1  0  0  8  0]
 [ 7  0  6  3  0  1  3  1  1  5]]

              precision    recall  f1-score   support

       blues       0.39      0.72      0.51        18
   classical       0.82      0.82      0.82        17
     country       0.27      0.35      0.31        17
       disco       0.43      0.48      0.45        21
      hiphop       0.56      0.43      0.49        21
        jazz       0.50      0.44      0.47        18
       metal       0.71      0.94      0.81        16
         pop       0.87  